In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, coalesce, lit, mean as _mean 
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier 
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline

# Инициализация SparkSession 
spark = SparkSession.builder \
    .appName("COVID19_ML_Model_RandomForest") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .config("spark.sql.parquet.datetimeRebaseModeInWrite", "LEGACY") \
    .getOrCreate()

optimized_parquet_path = "hdfs:///covid_dataset/metadata_optimized/"

# Путь в HDFS, где хранятся очищенные и оптимизированные метаданные в формате Parquet
df_ml = spark.read.parquet(optimized_parquet_path)

# Целевая переменная: is_covid (0 или 1)
# Признаки: sex, age_group, view, modality, RT_PCR_positive, survival, temperature, pO2_saturation, leukocyte_count, neutrophil_count, lymphocyte_count

# Список категориальных признаков, которые нужно индексировать и One-Hot-кодировать
categorical_features = [
    "sex",
    "age_group",
    "view",
    "modality",
    "RT_PCR_positive",
    "survival"
]

# Список числовых признаков
numeric_features = [
    "age_numeric",
    "temperature",
    "pO2_saturation",
    "leukocyte_count",
    "neutrophil_count",
    "lymphocyte_count"
]

# Валидация и заполнение пропусков в числовых признаках
for nf in numeric_features:
    if nf in df_ml.columns:
        if df_ml.filter(col(nf).isNull()).count() > 0:
            avg_val = df_ml.select(_mean(col(nf))).collect()[0][0]
            if avg_val is not None:
                df_ml = df_ml.withColumn(nf, col(nf).cast("double"))
                df_ml = df_ml.na.fill(avg_val, subset=[nf])
                print(f"Заполнены пропуски в {nf} средним {avg_val:.2f}")
            else:
                print(f"Предупреждение: Колонка {nf} содержит только NULL значения, невозможно заполнить средним. Заполняем 0.0.")
                df_ml = df_ml.withColumn(nf, lit(0.0).cast("double"))
    else:
        print(f"Предупреждение: Числовая колонка '{nf}' не найдена в DataFrame. Удаляем из списка.")
        numeric_features.remove(nf) # Удаляем несуществующие колонки из списка

# Создаем стадии Pipeline для StringIndexer и OneHotEncoderEstimator
indexers = [
    StringIndexer(inputCol=feature, outputCol=feature + "_indexed", handleInvalid="keep")
    for feature in categorical_features if feature in df_ml.columns
]

encoders = [
    OneHotEncoder(inputCol=feature + "_indexed", outputCol=feature + "_encoded")
    for feature in categorical_features if feature in df_ml.columns
]

assembler_inputs = [f.getOutputCol() for f in encoders] + numeric_features

if not assembler_inputs:
    raise ValueError("Не найдено действительных признаков для VectorAssembler после всех преобразований. Проверьте списки признаков и схему DataFrame.")

vector_assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="features")

# RandomForestClassifier
rf = RandomForestClassifier(featuresCol="features", labelCol="is_covid", numTrees=100, maxDepth=10, seed=42) 

# Создание Pipeline
pipeline = Pipeline(stages=indexers + encoders + [vector_assembler, rf]) 

print("Начинаем разделение данных на обучающую и тестовую выборки...")
# Разделение данных на обучающую 80% и тестовую 20% 
(training_data, test_data) = df_ml.randomSplit([0.8, 0.2], seed=42)

print(f"Обучающая выборка: {training_data.count()} строк")
print(f"Тестовая выборка: {test_data.count()} строк")

print("Начинаем обучение модели Random Forest...")
# Обучение модели
model = pipeline.fit(training_data)
print("Обучение модели Random Forest завершено.")

# Предсказания на тестовой выборке
print("Выполняем предсказания на тестовой выборке...")
predictions = model.transform(test_data)
predictions.select("patientid", "is_covid", "prediction", "probability").show(10, truncate=False)

# Оценка модели
print("Оцениваем производительность модели Random Forest...")
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="is_covid", metricName="areaUnderROC")
auc = evaluator.evaluate(predictions)
print(f"Area Under ROC (AUC) на тестовой выборке (Random Forest): {auc:.4f}")

# Важность признаков для Random Forest
try:
    rf_model = model.stages[-1] 
    if hasattr(rf_model, 'featureImportances'):
        print("\nВажность признаков (Random Forest):")
        importances = rf_model.featureImportances.toArray()
        # Сопоставим важность с именами признаков
        feature_names = vector_assembler.getInputCols()
        # Создаем список пар (признак, важность) и сортируем по важности
        feature_importance_pairs = sorted(zip(feature_names, importances), key=lambda x: x[1], reverse=True)
        for feature, importance in feature_importance_pairs:
            print(f"  {feature}: {importance:.4f}")
    else:
        print("\nВажность признаков недоступна для этой модели Random Forest.")
except Exception as e:
    print(f"\nНе удалось получить важность признаков Random Forest: {e}")

spark.stop()
print("SparkSession остановлена. Обучение и оценка ML-модели Random Forest завершены.")

Заполнены пропуски в leukocyte_count средним 5.02
Заполнены пропуски в neutrophil_count средним 5.31
Заполнены пропуски в lymphocyte_count средним 4.64
Начинаем разделение данных на обучающую и тестовую выборки...
Обучающая выборка: 797 строк
Тестовая выборка: 153 строк
Начинаем обучение модели Random Forest...
Обучение модели Random Forest завершено.
Выполняем предсказания на тестовой выборке...
+---------+--------+----------+-----------------------------------------+
|patientid|is_covid|prediction|probability                              |
+---------+--------+----------+-----------------------------------------+
|112      |1       |0.0       |[0.6978567191645906,0.3021432808354095]  |
|114      |1       |0.0       |[0.6099254138726917,0.3900745861273082]  |
|115      |1       |0.0       |[0.6099254138726917,0.3900745861273082]  |
|117      |1       |0.0       |[0.6248392613673083,0.3751607386326916]  |
|12       |1       |1.0       |[0.1102325694937897,0.8897674305062102]  |
|123      |1       |1.0       |[0.01882704707754584,0.9811729529224542] |
|129      |1       |1.0       |[0.01158898969242963,0.9884110103075703] |
|13       |1       |1.0       |[0.04336321248891425,0.9566367875110858] |
|134      |1       |1.0       |[0.016820204196537257,0.9831797958034627]|
|135      |1       |1.0       |[0.018642287027649144,0.9813577129723509]|
+---------+--------+----------+-----------------------------------------+
only showing top 10 rows

Оцениваем производительность модели Random Forest...
Area Under ROC (AUC) на тестовой выборке (Random Forest): 0.9054

Важность признаков (Random Forest):
  temperature: 0.0403
  lymphocyte_count: 0.0386
  view_encoded: 0.0177
  pO2_saturation: 0.0175
  survival_encoded: 0.0149
  age_numeric: 0.0110
  sex_encoded: 0.0106
  RT_PCR_positive_encoded: 0.0100
  age_group_encoded: 0.0088
  modality_encoded: 0.0067
  leukocyte_count: 0.0019
  neutrophil_count: 0.0001
SparkSession остановлена. Обучение и оценка ML-модели Random Forest завершены.